In [2]:
import numpy as np
import pandas as pd
import talib as ta
import yfinance as yf

In [3]:
df = yf.download("GOOG", start="2021-07-10", end="2022-07-06", interval="1h", progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-07-12 09:30:00,2596.669922,2602.840088,2592.000000,2599.695068,2599.695068,177791
2021-07-12 10:30:00,2599.989990,2610.767578,2598.889893,2606.820068,2606.820068,76378
2021-07-12 11:30:00,2605.939941,2610.800049,2602.510010,2603.620117,2603.620117,64613
2021-07-12 12:30:00,2603.564941,2606.489990,2601.570068,2602.969971,2602.969971,67478
2021-07-12 13:30:00,2602.945068,2606.475098,2598.000000,2604.669922,2604.669922,72896


In [4]:
df = df[["Open", "High", "Low", "Close", "Volume"]]
df.dropna(inplace=True)

df.head()

C:\Users\anish\AppData\Local\Temp\ipykernel_12576\1188362657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Open,High,Low,Close,Volume
Datetime,,,,,
2021-07-12 09:30:00,2596.669922,2602.840088,2592.000000,2599.695068,177791
2021-07-12 10:30:00,2599.989990,2610.767578,2598.889893,2606.820068,76378
2021-07-12 11:30:00,2605.939941,2610.800049,2602.510010,2603.620117,64613
2021-07-12 12:30:00,2603.564941,2606.489990,2601.570068,2602.969971,67478
2021-07-12 13:30:00,2602.945068,2606.475098,2598.000000,2604.669922,72896


In [5]:
from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

class RsiBreakout(TrailingStrategy):
    n1 = 40
    n2 = 200
    upper_bound = 60
    lower_bound = 40

    def init(self):
        super().init()
        self.set_trailing_sl(2.5)

        high = self.data.High
        low = self.data.Low                
        close = self.data.Close
        self.rsi = self.I(ta.RSI, close, self.n1)
        self.sma = self.I(ta.SMA, close, self.n2)
        self.atr = self.I(ta.ATR, high, low, close, 14)

    def next(self):
        super().next()
        close = self.data.Close[-1]

        if self.rsi[-1] < self.lower_bound and close<self.sma[-1] and not self.position:
            self.sl = self.data.Close[-1] - 2.5*self.atr[-1]
            self.buy(size=0.75, sl=self.sl)

        elif self.rsi[-1] > self.upper_bound and close>self.sma[-1] and not self.position:
            self.sl = self.data.Close[-1] + 2.5*self.atr[-1]
            self.sell(size=0.75, sl=self.sl)

e:\Programming\stock-trading\.conda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Programming\stock-trading\.conda\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 530 configurations.
  output = _optimize_grid()


In [ ]:
bt = Backtest(df, RsiBreakout,
            cash=100000, trade_on_close=True, exclusive_orders=True)

# stats = bt.optimize(n1=range(10, 80, 4), n2=range(40, 200, 5), maximize="Sharpe Ratio", constraint=lambda p: p.n1 < p.n2)

stats = bt.run()

In [6]:
print("OC-Range: ", (df["Close"].iloc[-1]-df["Close"].iloc[0])/df["Close"].iloc[0])
print("HL-Range: ", (np.max(df["Close"])-np.min(df["Close"]))/np.min(df["Close"]))

OC-Range:  -0.12333177221524723
HL-Range:  0.4586576948301164


In [7]:
stats

Start                     2021-07-12 09:30:00
End                       2022-07-05 15:30:00
Duration                    358 days 06:00:00
Exposure Time [%]                   23.023658
Equity Final [$]                117141.824868
Equity Peak [$]                 119764.671392
Return [%]                          17.141825
Buy & Hold Return [%]              -12.333177
Return (Ann.) [%]                   17.441136
Volatility (Ann.) [%]               12.367289
Sharpe Ratio                         1.410263
Sortino Ratio                        2.911868
Calmar Ratio                         3.412092
Max. Drawdown [%]                   -5.111567
Avg. Drawdown [%]                   -1.205484
Max. Drawdown Duration       97 days 21:00:00
Avg. Drawdown Duration       15 days 08:00:00
# Trades                                   23
Win Rate [%]                        60.869565
Best Trade [%]                      10.179319
Worst Trade [%]                     -2.777816
Avg. Trade [%]                    